In [2]:
from LipMovementClassifier import *

In [1]:
learningrates = [1e-3, 1e-4, 2e-4, 5e-4, 1e-5]
fakedatasets = ['featuresExtracted/deepfakesfeatures.npz', 'featuresExtracted/faceswapfeatures.npz', 'featuresExtracted/neuraltexturesfeatures.npz', 'featuresExtracted/face2facefeatures.npz']

In [3]:
for dataset in fakedatasets:
    for learningrate in learningrates:
        print("Dataset: ", dataset)
        print("Learning rate: ", learningrate)
        classifer = LipMovementClassifier(dataset, learning_rate=learningrate)
        classifer.train()
        classifer.test()

Epoch: 1, loss: 1.657
accuracy: 0.770
AUC: 0.833
Epoch: 2, loss: 0.293
accuracy: 0.883
AUC: 0.954
Epoch: 3, loss: 0.233
accuracy: 0.915
AUC: 0.973
Epoch: 4, loss: 0.153
accuracy: 0.943
AUC: 0.987
Epoch: 5, loss: 0.141
accuracy: 0.947
AUC: 0.989
Epoch: 6, loss: 0.105
accuracy: 0.960
AUC: 0.994
Epoch: 7, loss: 0.105
accuracy: 0.962
AUC: 0.994
Epoch: 8, loss: 0.076
accuracy: 0.970
AUC: 0.997
Epoch: 9, loss: 0.077
accuracy: 0.972
AUC: 0.997
Epoch: 10, loss: 0.055
accuracy: 0.981
AUC: 0.998
Time of Training: 803.612
Test Set Evaluation:
Test Videos Accuracy: 0.979
Test Videos AUC: 0.997
Test Videos Confusion Matrix:
[[139   1]
 [  5 135]]
Test Videos Classification Report:
              precision    recall  f1-score   support

         0.0       0.97      0.99      0.98       140
         1.0       0.99      0.96      0.98       140

    accuracy                           0.98       280
   macro avg       0.98      0.98      0.98       280
weighted avg       0.98      0.98      0.98       2